# モジュール

In [86]:
import pandas as pd
import re
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 30)

# 基本情報

#### 貸借情報

In [93]:
# スクレイピング対象のURL
url = 'https://kabutan.jp/stock/?code=5892'

# URLからデータを取得
response = requests.get(url)
response.encoding = 'utf-8'

# BeautifulSoupでHTMLを解析
soup = BeautifulSoup(response.text, 'html.parser')

# テキスト要素を抽出
text_elements = soup.get_text()

# 不要な空白や改行を除外
clean_text = re.sub(r'\s+', ' ', text_elements).strip() 

# 空白区切りで単語に分ける
words_list = clean_text.split()
words_list

['ｙｕｔｏｒｉ（ユトリ）【5892】株の基本情報｜株探（かぶたん）',
 '≫米株',
 '株探プレミアムに登録！',
 'ログイン',
 '日経平均',
 '15:00',
 '米ドル円',
 '00:03',
 'ＮＹダウ',
 '終値',
 '上海総合',
 '終値',
 '38,525.95',
 '+57.32',
 '154.50',
 '+0.48',
 '40,539.93',
 '-49.41',
 '2,879.29',
 '-12.54',
 '0.15%',
 '0.31%',
 '-0.13%',
 '-0.43%',
 'トップ',
 '市場ニュース',
 '決算速報',
 '株価注意報',
 '会社開示情報',
 '銘柄探検',
 '日本の株主',
 '市場マップ',
 '「株探」で配信したニュース記事を検索します',
 'JavaScriptが無効になっています。',
 '株探のすべての機能を利用するためには、JavaScriptを有効にしてください。',
 'トップ',
 'ユトリ(5892)',
 'ｙｕｔｏｒｉ(5892)',
 '基本情報',
 '信用',
 '証券取引所が指定する制度信用銘柄のうち、買建(信用買い)のみができる銘柄',
 '株価20分ディレイ',
 '→',
 'リアルタイムに変更',
 '5892',
 'ｙｕｔｏｒｉ',
 '東証Ｇ',
 '15:00',
 '1,519円',
 '前日比',
 '-20',
 '-1.30%',
 'PTS',
 '－円',
 '－',
 '業績',
 '小売業',
 '単位',
 '100株',
 'PER',
 'PBR',
 '利回り',
 '信用倍率',
 '26.0倍',
 '10.94倍',
 '－％',
 '－倍',
 '時価総額',
 '71.4億円',
 '比較される銘柄',
 'コックス,',
 'キューブ,',
 'ティーライフ',
 '基本情報',
 'チャート',
 '時系列',
 'ニュース',
 '決算',
 '大株主',
 '決算発表予定日',
 '2024/08/13',
 'こちらは株探プレミアム',
 '「日本株プラン」',
 'または「日米セットプラン」限定コンテンツ',
 'です。プランをご契約して見ることができます。',
 

In [94]:
# それぞれの単語のインデックスを取得
realtime_index = words_list.index("リアルタイムに変更")
hikaku_index = words_list.index("比較される銘柄")

# インデックス番号の間にあるワードを抽出
taishaku_info = words_list[realtime_index + 1 : hikaku_index]

# 最初の3要素を抜き出す
first_four = taishaku_info[:3]

# 「業績」というワードの次の要素を抜き出す
index_of_gyouseki = taishaku_info.index('業績')
gyousyu = taishaku_info[index_of_gyouseki + 1]

# 最後の6要素を抜き出す
last_six = taishaku_info[-6:]
last_six.remove("時価総額")

# 全てを一つのリストに格納
cleaned_taishaku_info = first_four + [gyousyu] + last_six
cleaned_taishaku_info

columns = ['証券コード', '銘柄', '市場', '業種', 'PER', 'PBR', '利回り', '信用倍率', '時価総額']

taishaku_df = pd.DataFrame([cleaned_taishaku_info], columns=columns)
taishaku_df

,証券コード,銘柄,市場,業種,PER,PBR,利回り,信用倍率,時価総額
0,5892,ｙｕｔｏｒｉ,東証Ｇ,小売業,26.0倍,10.94倍,－％,－倍,71.4億円


#### 株価トレンド・業績推移

In [95]:
# 目的の単語
gonitisen = "5日線"
happyoubi = "発表日"
tansin = "直近の決算短信"

if gonitisen in words_list:
    index_gonitisen = words_list.index(gonitisen)
    trend_list = words_list[index_gonitisen + 4 : index_gonitisen + 8]  # インデックス+5から+8までを含む
else:
    nitisen_list = []

if happyoubi in words_list and tansin in words_list:
    index_happyoubi = words_list.index(happyoubi)
    index_tansin = words_list.index(tansin)
    if index_happyoubi < index_tansin:
        suii_list = words_list[index_happyoubi + 1 : index_tansin] 
    else:
        suii_list = []
else:
    suii_list = []
    
suii_list.remove('予')

print("trend_list:", trend_list)
print("suii_list:", suii_list)

trend_list: ['+0.29％', '-1.82％', '+7.36％', '+17.33％']
suii_list: ['2023.03*', '24.7', '-0.5', '-0.7', '-15.4', '0.0', '－', '2024.03', '43.2', '3.8', '2.2', '49.9', '0.0', '24/05/14', '2025.03', '56.1', '4.9', '2.7', '58.3', '0.0', '24/05/14', '前期比(%)', '+30.0', '+30.2', '+21.8', '+16.9']


In [96]:
trend_df = pd.DataFrame([nitisen_list], columns=['5日線', '25日線', '75日線', '200日線'])

# suii_listの長さを7の倍数に調整する
while len(suii_list) % 7 != 0:
    suii_list.append(None)  # 足りない部分をNoneで埋める
suii_array = np.array(suii_list).reshape(-1, 7)

# データフレームに変換し、列名を設定
suii_df = pd.DataFrame(suii_array, columns=['決算期', '売上高', '経常益', '最終益', '１株益', '１株配', '発表日'])
display(trend_df)
display(suii_df)

ValueError: 4 columns passed, passed data had 0 columns

In [90]:
DATA_DIR = '/Users/yuseiito/Desktop/stock/data/'
kihon_df = pd.concat([taishaku_df, trend_df], axis=1)
kihon_df.to_csv(f'{DATA_DIR}/basic_info.csv', index=False)
kihon_df

,証券コード,銘柄,市場,業種,PER,PBR,利回り,信用倍率,時価総額,5日線,25日線,75日線,200日線
0,9997,ベルーナ,東証Ｐ,小売業,9.3倍,0.55倍,3.76％,16.59倍,751億円,0.00％,-1.45％,+4.09％,+5.27％


In [91]:
shouken_code = pd.DataFrame({'証券コード': np.repeat(kihon_df['証券コード'].values, len(suii_df))})
# suii_dfとkihon_df_repeatedを結合
suii_code_df = pd.concat([shouken_code.reset_index(drop=True), suii_df.reset_index(drop=True)], axis=1)
suii_code_df.to_csv(f'{DATA_DIR}/performance_trend.csv', index=False)
suii_code_df

,証券コード,決算期,売上高,経常益,最終益,１株益,１株配,発表日
0,9997,2023.03,"2,123",124,74.2,76.7,20.0,23/05/12
1,9997,2024.03,"2,082",118,58.4,60.4,20.5,24/05/13
2,9997,2025.03,"2,170",125,80.0,82.7,29.0,24/05/13
3,9997,前期比(%),+4.2,+5.7,+37.0,+37.0,None,None
